In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

input_folder = os.path.join(os.getcwd(),'raw')
if(not os.path.exists(input_folder)):
    os.mkdir(input_folder)

output_folder = os.path.join(os.getcwd(),'generated')
if(not os.path.exists(output_folder)):
    os.mkdir(output_folder)

In [7]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# Full example url https://www.sec.gov/files/data/fails-deliver-data/cnsfails202110b.zip
base_url = "https://www.sec.gov/files/data/fails-deliver-data/"
file_prefix = "cnsfails"
start_year = 2021
start_month = 7
end_year = 2021
end_month = 10

def download_and_unzip(url):
    print("Download " + url)
    resp = urlopen(url)
    zipfile = ZipFile(BytesIO(resp.read()))
    zipfile.extractall(path=input_folder)

for year in range(start_year, end_year+1):
    start = start_month if (year == start_year) else 1
    end = end_month if (year == end_year) else 12
    for month in range(start, end+1):
        download_and_unzip(f"{base_url}{file_prefix}{year}{month:02d}a.zip")
        download_and_unzip(f"{base_url}{file_prefix}{year}{month:02d}b.zip")



Download https://www.sec.gov/files/data/fails-deliver-data/cnsfails202107a.zip
Download https://www.sec.gov/files/data/fails-deliver-data/cnsfails202107b.zip
Download https://www.sec.gov/files/data/fails-deliver-data/cnsfails202108a.zip
Download https://www.sec.gov/files/data/fails-deliver-data/cnsfails202108b.zip
Download https://www.sec.gov/files/data/fails-deliver-data/cnsfails202109a.zip
Download https://www.sec.gov/files/data/fails-deliver-data/cnsfails202109b.zip
Download https://www.sec.gov/files/data/fails-deliver-data/cnsfails202110a.zip
Download https://www.sec.gov/files/data/fails-deliver-data/cnsfails202110b.zip


In [17]:
def read_csv(input_folder, name):
    li = []

    for path in Path(input_folder).rglob(f'*{name}*.txt'):
        print(f"Read csv {path}")
        csv_df = pd.read_csv(path, index_col=None, header=0, sep ='|', encoding = "ISO-8859-1")
        li.append(csv_df)

    df = pd.concat(li, axis=0, ignore_index=True)
    return df

df = read_csv(input_folder, 'cnsfails')
df.head()

Read csv c:\Data\Documents\Python\gme\raw\cnsfails202107a.txt
Read csv c:\Data\Documents\Python\gme\raw\cnsfails202107b.txt
Read csv c:\Data\Documents\Python\gme\raw\cnsfails202108a.txt
Read csv c:\Data\Documents\Python\gme\raw\cnsfails202108b.txt
Read csv c:\Data\Documents\Python\gme\raw\cnsfails202109a.txt
Read csv c:\Data\Documents\Python\gme\raw\cnsfails202109b.txt
Read csv c:\Data\Documents\Python\gme\raw\cnsfails202110a.txt


,SETTLEMENT DATE,CUSIP,SYMBOL,QUANTITY (FAILS),DESCRIPTION,PRICE
0,20210701,B38564108,EURN,190881.0,EURONAV NV ANTWERPEN (BELGIUM),9.32
1,20210701,D1668R123,DDAIF,340460.0,DAIMLER AG,89.90
2,20210701,D18190898,DB,3551.0,DEUTSCHE BANK AG NAMEN AKT (DE,13.08
3,20210701,G0083D120,ACEV,45.0,ACE CONVERGENCE ACQUISITION CO,9.96
4,20210701,G0083E110,ACBAW,3050.0,ACE GLOBAL BUSINESS ACQSTN LTD,0.49
